In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.utils import np_utils
from keras.layers import BatchNormalization, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import tensorflow as tf
print(tf.__version__)

2.6.0


In [2]:
df_train = pd.read_csv(r'C:\Users\Tejas\Desktop\Capstone\DatasetsCleaned\UniformClasses\V2\train\ohc_train.csv', sep='\t')

In [3]:
df_test = pd.read_csv(r'C:\Users\Tejas\Desktop\Capstone\DatasetsCleaned\UniformClasses\V2\test\ohc_test.csv', sep='\t')

In [4]:
df_train

,Text,Class
0,\tif want big buck tri moba,NOT
1,\t\t\tthe democrat republican polit flip old ...,NOT
2,\ta year back twat protest japanes cultur exh...,NOT
3,\t\tbut give cheater money cuz vagina fine le...,NOT
4,\thow reboot franchis less content origin gam...,NOT
...,...,...
8280,\t\tare still peopl pretend trump isnt retard...,NOT
8281,\t\t\t\t\tjust look ugli butch dyke look one,NOT
8282,\t\tyeah saw one later bitter bitter human,NOT
8283,\t\t\t\tnope call finotci us one capitalist ...,NOT


In [5]:
def getLemmText(text):
    tokens=word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens=[lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)
 
df_train['Text'] = list(map(getLemmText,df_train['Text']))
df_test['Text'] = list(map(getLemmText,df_test['Text']))

In [6]:
def getStemmText(text):
    tokens=word_tokenize(text)
    ps = PorterStemmer()
    tokens=[ps.stem(word) for word in tokens]
    return ' '.join(tokens)

df_train['Text'] = list(map(getStemmText,df_train['Text']))
df_test['Text'] = list(map(getStemmText,df_test['Text']))


In [7]:
xtrain = df_train['Text']
ytrain = df_train['Class']
xtest = df_test['Text']
ytest = df_test['Class']

print(len(xtrain), len(ytrain))
print(len(xtest), len(ytest))

8285 8285
2090 2090


In [ ]:
EMBEDDING_DIMENSION = 64
VOCABULARY_SIZE = 2000
MAX_LENGTH = 100
OOV_TOK = '<OOV>'
TRUNCATE_TYPE = 'post'
PADDING_TYPE = 'post'

In [8]:
MAX_NB_WORDS=75000
MAX_SEQUENCE_LENGTH=500
EMBEDDING_DIM=50

In [9]:
# tokenizer = Tokenizer(num_words=VOCABULARY_SIZE, oov_token=OOV_TOK)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(list(xtrain) + list(xtest))

In [10]:
xtrain_sequences = tokenizer.texts_to_sequences(xtrain)
xtest_sequences = tokenizer.texts_to_sequences(xtest)
word_index = tokenizer.word_index
print('Vocabulary size:', len(word_index))
dict(list(word_index.items())[0:10])

Vocabulary size: 21165


{'like': 1,
 'peopl': 2,
 'get': 3,
 'fuck': 4,
 'dont': 5,
 'woman': 6,
 'one': 7,
 'think': 8,
 'would': 9,
 'say': 10}

In [ ]:
# xtrain_pad = sequence.pad_sequences(xtrain_sequences, maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNCATE_TYPE)
# xtest_pad = sequence.pad_sequences(xtest_sequences, maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNCATE_TYPE)

In [11]:
xtrain_pad = sequence.pad_sequences(xtrain_sequences, maxlen=MAX_SEQUENCE_LENGTH)
xtest_pad = sequence.pad_sequences(xtest_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [13]:
embeddings_index = {}
f = open(r"C:\Users\Tejas\Desktop\Capstone\hate-speech-project\glove.6B.50d.txt", encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
    except:
        pass
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        if len(embedding_matrix[i]) != len(embedding_vector):
            print("could not broadcast input array from shape", str(len(embedding_matrix[i])),
                  "into shape", str(len(embedding_vector)), " Please make sure your"
                                                            " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
            exit(1)
        embedding_matrix[i] = embedding_vector

In [ ]:
xtrain_sequences = np.array(xtrain_sequences)

In [14]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(list(ytrain))
training_label_seq = np.array(label_tokenizer.texts_to_sequences(ytrain))
test_label_seq = np.array(label_tokenizer.texts_to_sequences(ytest))
print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

[1]
[1]
[1]
(8285, 1)


In [15]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_article(text):
 return ''.join([reverse_word_index.get(i, '?') for i in text])
print(decode_article(xtrain_pad[11]))

???????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????yousoundlikepedophildenialadmissthinkanyonchildknowdamnwellyouvattractnumerteenagageyourtotalfuckhypocritanywayreallihopeyourwomanyourdudecutballlongtimeago


In [18]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
# model.add(Embedding(len(word_index) + 1,
#                      EMBEDDING_DIMENSION))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(EMBEDDING_DIM, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 50)           1058300   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 500, 50)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               40400     
_________________________________________________________________
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)               

In [ ]:
for x in xtrain_sequences:
    x = np.array(x)

In [ ]:
for x in xtest_sequences:
    x = np.array(x)

In [ ]:
xtrain_sequences

In [20]:
num_epochs = 10
history = model.fit(xtrain_pad, training_label_seq, epochs=num_epochs, validation_data=(xtest_pad, test_label_seq), verbose=1)

Epoch 1/10
  7/259 [..............................] - ETA: 1:11:25 - loss: nan - accuracy: 0.0491

KeyboardInterrupt: 